In [1]:
import cv2
import numpy as np
import serial
import time

# --- КОНФИГУРАЦИЈА ---
SERIAL_PORT = 'COM5'
BAUD_RATE = 115200
CAMERA_INDEX = 1
STOP_COMMAND = 90
DEADZONE_PIXELS = 18  

# Твоите калибрирани бои
LOWER_COLOR = np.array([0, 92, 188])  
UPPER_COLOR = np.array([98, 255, 255])

# --- PID И ФИЛТЕР ПАРАМЕТРИ ---
KP_VAL = 0.3      
KI_VAL = 0.003     # Малку ја зголемив за да има ефект, бидејќи 0.0001 е премногу мала
KD_VAL = 0.13      
ALPHA = 0.6        

# --- ANTI-WINDUP ГРАНИЦА ---
I_LIMIT = 10        # Максимален придонес на интегралот во степени

# ЛОГИКА ЗА ФРЕЈМОВИ
MIN_FRAMES_TO_START = 5
MAX_FRAMES_LOST = 5

# --- ИНИЦИЈАЛИЗАЦИЈА ---
try:
    arduino = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.001)
    time.sleep(2)
    print("MG995 Системот е подготвен!")
except Exception as e:
    print(f"Грешка со Arduino: {e}")
    exit()

cap = cv2.VideoCapture(CAMERA_INDEX)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

# Променливи за состојба
prev_error_x, prev_error_y = 0, 0
integral_x, integral_y = 0, 0  # <--- НОВО: Интегрални акумулатори
smooth_ball_x, smooth_ball_y = 160, 120
prev_time = time.time()
is_tracking = False
detection_counter = 0
lost_counter = 0

print("Програмата работи. Притиснете 'q' за излез.")

try:
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, LOWER_COLOR, UPPER_COLOR)
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ball_x, ball_y = None, None
        if contours:
            c = max(contours, key=cv2.contourArea)
            if cv2.contourArea(c) > 80:
                M = cv2.moments(c)
                if M["m00"] != 0:
                    ball_x = int(M["m10"] / M["m00"])
                    ball_y = int(M["m01"] / M["m00"])

        if ball_x is not None:
            detection_counter += 1
            lost_counter = 0
            if detection_counter >= MIN_FRAMES_TO_START:
                is_tracking = True
        else:
            lost_counter += 1
            detection_counter = 0
            if lost_counter >= MAX_FRAMES_LOST:
                is_tracking = False

        curr_time = time.time()
        dt = curr_time - prev_time
        if dt < 0.001: dt = 0.001

        if is_tracking and ball_x is not None:
            smooth_ball_x = (ALPHA * ball_x) + ((1 - ALPHA) * smooth_ball_x)
            smooth_ball_y = (ALPHA * ball_y) + ((1 - ALPHA) * smooth_ball_y)
            
            error_x = 160 - smooth_ball_x
            error_y = 120 - smooth_ball_y
            
            if abs(error_x) < DEADZONE_PIXELS and abs(error_y) < DEADZONE_PIXELS:
                pwm_x, pwm_y = 90, 90
                integral_x = integral_y = 0 # Ресет на интеграл во центар
            else:
                # ПРЕСМЕТКА НА ИНТЕГРАЛ СО ЛИМИТ (Anti-Windup)
                integral_x += error_x * dt
                integral_y += error_y * dt
                
                # Лимитирање на интегралот
                integral_x = np.clip(integral_x, -I_LIMIT/KI_VAL, I_LIMIT/KI_VAL)
                integral_y = np.clip(integral_y, -I_LIMIT/KI_VAL, I_LIMIT/KI_VAL)

                # PID X
                p_x = error_x * KP_VAL
                i_x = integral_x * KI_VAL
                d_x = ((error_x - prev_error_x) / dt) * KD_VAL
                pwm_x = int(STOP_COMMAND + p_x + i_x + d_x)
                
                # PID Y
                p_y = error_y * KP_VAL
                i_y = integral_y * KI_VAL
                d_y = ((error_y - prev_error_y) / dt) * KD_VAL
                pwm_y = int(STOP_COMMAND + p_y + i_y + d_y)

            pwm_x = np.clip(pwm_x, 65, 115)
            pwm_y = np.clip(pwm_y, 65, 115)
            
            arduino.write(f"{pwm_x},{pwm_y}\n".encode())
            prev_error_x, prev_error_y = error_x, error_y
        else:
            arduino.write(b"90,90\n")
            integral_x = integral_y = 0 # Ресет кога нема детекција
            prev_error_x = prev_error_y = 0
            
        prev_time = curr_time
        
        cv2.circle(frame, (int(smooth_ball_x), int(smooth_ball_y)), 10, (0, 255, 0), 2)
        cv2.circle(frame, (160, 120), DEADZONE_PIXELS, (255, 0, 0), 1)
        
        cv2.imshow("Stable PID Control", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break

finally:
    print("Затворање...")
    arduino.write(b"90,90\n")
    arduino.close()
    cap.release()
    cv2.destroyAllWindows()

MG995 Системот е подготвен!
Програмата работи. Притиснете 'q' за излез.
Затворање...
